# One BLOCK at a time

## Recap

Our initial message:

In [1]:
message = "Hi!"
message

'Hi!'

We have our message converted from text to ascii to binary:

In [2]:
binary_message = ''.join(format(ord(char), '08b') for char in message)
binary_message

'010010000110100100100001'

We have our padded message.

In [3]:
def pad_binary(binary_message,
               block_size=512,
               length_field=64):

    # Add the '1' bit
    padded = binary_message + '1'

    # Calculate Zeros Needed
    msg_size_on_final_block = (len(padded) % block_size)
    zeros_needed = block_size - msg_size_on_final_block - length_field

    # If there are not enough zeros, add another block w/ zeros
    if zeros_needed < 0:
        zeros_needed += 512
    
    # Otherwise, append the zeros needed
    padded += '0' * zeros_needed

    # Add 64-bit message length to the end, padded with zeros
    msg_length = len(binary_message)
    length_bits = format(msg_length, '064b')
    final_padded = padded + length_bits

    return final_padded

pad_binary(binary_message)

'01001000011010010010000110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011000'

## Messages Longer than One Block

In [18]:
message = "This is a longer message that will require more than one 512-bit block, so we will need a function to break them down."
binary_message = ''.join(format(ord(char), '08b') for char in message)
padded_message = pad_binary(binary_message)

The padded message is too long. Needs to be broken down into 512-bit blocks.

In [24]:
def divide_into_blocks(binary_string, block_size=512):
    """
    Divides a binary string into blocks of a specified size.

    Args:
        binary_string (str): The binary string to be divided.
        block_size (int): The size of each block in bits (default is 512).

    Returns:
        list: A list of binary string blocks.
    """
    # Ensure the binary string length is a multiple of 8
    if len(binary_string) % 8 != 0:
        raise ValueError("Binary string length must be a multiple of 8")

    # Split the binary string into blocks
    blocks = []
    for i in range(0, len(binary_string), block_size):
        blocks.append(binary_string[i:i+block_size])
    
    return blocks

And here we can see the blocks in both binary format.

In [26]:
# Divide the padded binary string into 512-bit blocks
blocks = divide_into_blocks(padded_message)

# Print the blocks in binary format
print("Blocks in Binary Format:")
for i, block in enumerate(blocks):
    print(f"Block {i + 1}: {block}")


Blocks in Binary Format:
Block 1: 01010100011010000110100101110011001000000110100101110011001000000110000100100000011011000110111101101110011001110110010101110010001000000110110101100101011100110111001101100001011001110110010100100000011101000110100001100001011101000010000001110111011010010110110001101100001000000111001001100101011100010111010101101001011100100110010100100000011011010110111101110010011001010010000001110100011010000110000101101110001000000110111101101110011001010010000000110101001100010011001000101101011000100110100101110100
Block 2: 001000000110001001101100011011110110001101101011001011000010000001110011011011110010000001110111011001010010000001110111011010010110110001101100001000000110111001100101011001010110010000100000011000010010000001100110011101010110111001100011011101000110100101101111011011100010000001110100011011110010000001100010011100100110010101100001011010110010000001110100011010000110010101101101001000000110010001101111011101110110111000101110100000000000

But we can also see it in byte string format.

In [28]:
# Convert binary blocks to byte strings and print
print("\nBlocks in Byte String Format:")
for i, block in enumerate(blocks):
    byte_array = bytearray()
    for j in range(0, len(block), 8):
        byte = block[j:j+8]
        byte_array.append(int(byte, 2))
    byte_string = bytes(byte_array)
    print(f"Block {i + 1}: {byte_string}")


Blocks in Byte String Format:
Block 1: b'This is a longer message that will require more than one 512-bit'
Block 2: b' block, so we will need a function to break them down.\x80\x00\x00\x00\x00\x00\x00\x00\x03\xb0'


#### What's bytestring format?

Reasons to Use Byte Strings
Standard Library Functions:

Many standard library functions and cryptographic operations expect data in byte string format. For example, the struct.unpack function used in the prepare_message_schedule function requires byte strings.
Efficiency:

Byte strings are more memory-efficient and faster to process compared to binary strings. Each byte in a byte string represents 8 bits, whereas a binary string uses one character per bit.
Compatibility:

Byte strings are the standard way to handle binary data in Python and other programming languages. This ensures compatibility with various libraries and tools.

In [ ]:
blocks